In [ ]:
import itertools

import numpy as np
import pandas as pd

from meterstick import confidence_interval_display

In [ ]:
np.random.seed(42)
metrics = ('Click', 'Latency', 'a very very very looooooooooong metric')
ctrl_id = 42
platform = ('Mobile', 'Desktop', 'Tablet')
ctrl_vals = [137, 28, 999.9, 158, 40, 6.66666, -10, -20.1, 33]
expr_ids = (42, 222, 666, 'Experiment Foo')
n_row = len(metrics) * len(platform) * len(expr_ids)
test_df = pd.DataFrame(
    itertools.product(expr_ids, platform, metrics),
    columns=[
        'Experiment_Id',
        'Platform',
        'Metric',
    ])
test_df['Control_Value'] = ctrl_vals * (n_row // len(ctrl_vals))
test_df['Ratio'] = 2 * (np.random.rand(n_row) - 0.5)
test_df['CI_Range'] = 2 * np.random.rand(n_row)
test_df['Value'] = test_df.Control_Value * (test_df.Ratio / 100 + 1)
test_df['Country'] = 'US'
ctrl_id = 42
test_df.loc[test_df.Experiment_Id == ctrl_id,
            'Value'] = test_df.loc[test_df.Experiment_Id == ctrl_id,
                                   'Control_Value']

In [ ]:
confidence_interval_display.render(test_df, dims=['Country', 'Platform'])

Dimensions,Click,Latency,a very very very looooooooooong metric
42US * Desktop,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
Experiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
42US * Mobile,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
Experiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
42US * Tablet,"-10.0000-0.88%[-0.92, -0.85] %","-20.10000.73%[-0.18, 1.64] %","33.00000.20%[-0.06, 0.46] %"
222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# You can manually specify the columns.
test_df_copy = test_df.copy()
test_df_copy.rename(
    columns={
        'Experiment_Id': 'expr',
        'Metric': 'metric',
        'Control_Value': 'control',
        'Ratio': 'ratio',
        'CI_Range': 'ci',
        'Value': 'val'
    },
    inplace=True)
confidence_interval_display.render(
    test_df_copy,
    dims=['Country', 'Platform'],
    metric='metric',
    ratio='ratio',
    value='val',
    ci_range='ci',
    control_value='control',
    expr_id='expr')

Dimensions,Click,Latency,a very very very looooooooooong metric
42US * Desktop,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
Experiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
42US * Mobile,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
Experiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
42US * Tablet,"-10.0000-0.88%[-0.92, -0.85] %","-20.10000.73%[-0.18, 1.64] %","33.00000.20%[-0.06, 0.46] %"
222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
test_df_copy = test_df.copy()
test_df_copy['Control_Id'] = None
confidence_interval_display.render(test_df_copy, dims=['Country', 'Platform'])

Dimensions,Click,Latency,a very very very looooooooooong metric
Not Control42US * Desktop,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
Not Control222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
Not Control666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
Not ControlExperiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
Not Control42US * Mobile,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
Not Control222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
Not Control666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
Not ControlExperiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
Not Control42US * Tablet,"-10.0000-0.88%[-0.92, -0.85] %","-20.10000.73%[-0.18, 1.64] %","33.00000.20%[-0.06, 0.46] %"
Not Control222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# If you don't want to aggregate your dimensions...
confidence_interval_display.render(
    test_df,
    dims=['Country', 'Platform'],
    aggregate_dimensions=False)

Country,Platform,Experiment_Id,Click,Latency,a very very very looooooooooong metric
US,Desktop,42,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
US,Desktop,222,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
US,Desktop,666,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
US,Desktop,Experiment Foo,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
US,Mobile,42,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
US,Mobile,222,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
US,Mobile,666,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
US,Mobile,Experiment Foo,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
US,Tablet,42,"-10.0000-0.88%[-0.92, -0.85] %","-20.10000.73%[-0.18, 1.64] %","33.00000.20%[-0.06, 0.46] %"
US,Tablet,222,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# By default we look for Dim_1, Dim2, ... as dimension columns.
test_df.rename(columns={'Country': 'Dim_1', 'Platform': 'Dim_2'}, inplace=True)
confidence_interval_display.render(test_df)

Dimensions,Click,Latency,a very very very looooooooooong metric
42US * Desktop,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
Experiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
42US * Mobile,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
Experiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
42US * Tablet,"-10.0000-0.88%[-0.92, -0.85] %","-20.10000.73%[-0.18, 1.64] %","33.00000.20%[-0.06, 0.46] %"
222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# You can set control experiment. We'll only display its Value.
confidence_interval_display.render(test_df, ctrl_id=ctrl_id)

Dimensions,Click,Latency,a very very very looooooooooong metric
Control42US * Desktop,158.0000,40.0000,6.6667
Not Control222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
Not Control666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
Not ControlExperiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
Control42US * Mobile,137.0000,28.0000,999.9000
Not Control222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
Not Control666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
Not ControlExperiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
Control42US * Tablet,-10.0000,-20.1000,33.0000
Not Control222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# Hide the controls if it doesn't matter to you.
confidence_interval_display.render(test_df, ctrl_id=ctrl_id, show_control=False)

Dimensions,Click,Latency,a very very very looooooooooong metric
Not Control222US * Desktop,"-0.66%[0.12, 1.21] %","--0.58%[-0.76, -0.39] %","--0.64%[-1.61, 0.33] %"
Not Control666US * Desktop,"--0.72%[-0.92, -0.53] %","--0.42%[-0.46, -0.37] %","--0.27%[-0.59, 0.06] %"
Not ControlExperiment FooUS * Desktop,"-0.22%[0.07, 0.36] %","--0.66%[-1.46, 0.14] %","--0.87%[-0.94, -0.80] %"
Not Control222US * Mobile,"-0.42%[-0.25, 1.08] %","--0.96%[-1.27, -0.65] %","-0.94%[0.42, 1.46] %"
Not Control666US * Mobile,"--0.14%[-0.73, 0.46] %","--0.42%[-1.34, 0.50] %","-0.22%[0.14, 0.31] %"
Not ControlExperiment FooUS * Mobile,"-0.03%[-0.33, 0.39] %","-0.18%[-0.10, 0.47] %","--0.91%[-1.45, -0.36] %"
Not Control222US * Tablet,"--0.63%[-1.41, 0.14] %","--0.39%[-1.33, 0.55] %","-0.05%[-0.85, 0.94] %"
Not Control666US * Tablet,"--0.09%[-0.48, 0.30] %","-0.57%[0.30, 0.84] %","--0.60%[-1.43, 0.23] %"
Not ControlExperiment FooUS * Tablet,"-0.90%[-0.09, 1.88] %","-0.93%[0.16, 1.70] %","-0.62%[0.42, 0.82] %"


In [ ]:
# You can also have multiple controls.
confidence_interval_display.render(
    test_df,
    ctrl_id={
        42: [222],
        666: ['Experiment Foo']
    })

Dimensions,Click,Latency,a very very very looooooooooong metric
Control42US * Desktop,158.0000,40.0000,6.6667
Not Control222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
Control666US * Desktop,156.8608,39.8337,6.6488
Not ControlExperiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
Control42US * Mobile,137.0000,28.0000,999.9000
Not Control222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
Control666US * Mobile,136.8135,27.8831,1002.1368
Not ControlExperiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
Control42US * Tablet,-10.0000,-20.1000,33.0000
Not Control222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# Auto determines control values if control rows are missing. Your control rows
# are supposed to have the same values in the Control_Value and Value columns.
# If not, we skip. To demonstrate, here we introduce inconsistency in 'Click'.
df = test_df.loc[test_df.Experiment_Id != 42].copy()
df.loc[(df.Experiment_Id == 222) & (df.Metric == 'Click'), 'Control_Value'] = -1
confidence_interval_display.render(
    df,
    ctrl_id=42,
    auto_decide_control_vals=True)

Dimensions,Click,Latency,a very very very looooooooooong metric
Control42US * Desktop,N/A,40.0000,6.6667
Not Control222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
Not Control666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
Not ControlExperiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
Control42US * Mobile,N/A,28.0000,999.9000
Not Control222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
Not Control666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
Not ControlExperiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
Control42US * Tablet,N/A,-20.1000,33.0000
Not Control222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# You can flip the coloring scheme for some metrics.
confidence_interval_display.render(test_df, flip_color=['Latency'])

Dimensions,Click,Latency,a very very very looooooooooong metric
42US * Desktop,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
Experiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
42US * Mobile,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
Experiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
42US * Tablet,"-10.0000-0.88%[-0.92, -0.85] %","-20.10000.73%[-0.18, 1.64] %","33.00000.20%[-0.06, 0.46] %"
222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# You can add descriptions for your experiments.
test_df['Description'] = ''
test_df.loc[test_df['Experiment_Id'] == 42, 'Description'] = 'The Answer'
# You can even use raw html.
test_df.loc[test_df['Experiment_Id'] == 666,
            'Description'] = '<b>Bold</b>'
confidence_interval_display.render(test_df)

Dimensions,Click,Latency,a very very very looooooooooong metric
The Answer42US * Desktop,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
Bold666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
Experiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
The Answer42US * Mobile,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
Bold666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
Experiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
The Answer42US * Tablet,"-10.0000-0.88%[-0.92, -0.85] %","-20.10000.73%[-0.18, 1.64] %","33.00000.20%[-0.06, 0.46] %"
222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# You can customize the formatting of values. By default there are 'percent',
# 'absolute' and 'pp'. You can also provide your own formating string template.
confidence_interval_display.render(
    test_df,
    metric_formats={
        'Ratio': 'pp',
        'Value': 'percent'
    })

Dimensions,Click,Latency,a very very very looooooooooong metric
The Answer42US * Desktop,"15800.00%0.20pp[-0.24, 0.64] pp","4000.00%-0.69pp[-0.81, -0.57] pp","666.67%-0.69pp[-1.18, -0.19] pp"
222US * Desktop,"15905.05%0.66pp[0.12, 1.21] pp","3976.99%-0.58pp[-0.76, -0.39] pp","662.42%-0.64pp[-1.61, 0.33] pp"
Bold666US * Desktop,"15686.08%-0.72pp[-0.92, -0.53] pp","3983.37%-0.42pp[-0.46, -0.37] pp","664.88%-0.27pp[-0.59, 0.06] pp"
Experiment FooUS * Desktop,"15833.98%0.22pp[0.07, 0.36] pp","3973.64%-0.66pp[-1.46, 0.14] pp","660.87%-0.87pp[-0.94, -0.80] pp"
The Answer42US * Mobile,"13700.00%-0.25pp[-0.56, 0.05] pp","2800.00%0.90pp[0.80, 1.00] pp","99990.00%0.46pp[-0.22, 1.15] pp"
222US * Mobile,"13757.01%0.42pp[-0.25, 1.08] pp","2773.15%-0.96pp[-1.27, -0.65] pp","100929.73%0.94pp[0.42, 1.46] pp"
Bold666US * Mobile,"13681.35%-0.14pp[-0.73, 0.46] pp","2788.31%-0.42pp[-1.34, 0.50] pp","100213.68%0.22pp[0.14, 0.31] pp"
Experiment FooUS * Mobile,"13703.90%0.03pp[-0.33, 0.39] pp","2805.18%0.18pp[-0.10, 0.47] pp","99082.99%-0.91pp[-1.45, -0.36] pp"
The Answer42US * Tablet,"-1000.00%-0.88pp[-0.92, -0.85] pp","-2010.00%0.73pp[-0.18, 1.64] pp","3300.00%0.20pp[-0.06, 0.46] pp"
222US * Tablet,"-993.67%-0.63pp[-1.41, 0.14] pp","-2002.13%-0.39pp[-1.33, 0.55] pp","3301.63%0.05pp[-0.85, 0.94] pp"


In [ ]:
# You don't need Value if you don't want to show_control.
test_df2 = test_df.copy()
del test_df2['Value']
confidence_interval_display.render(test_df2, show_control=False)

Dimensions,Click,Latency,a very very very looooooooooong metric
The Answer42US * Desktop,"-0.20%[-0.24, 0.64] %","--0.69%[-0.81, -0.57] %","--0.69%[-1.18, -0.19] %"
222US * Desktop,"-0.66%[0.12, 1.21] %","--0.58%[-0.76, -0.39] %","--0.64%[-1.61, 0.33] %"
Bold666US * Desktop,"--0.72%[-0.92, -0.53] %","--0.42%[-0.46, -0.37] %","--0.27%[-0.59, 0.06] %"
Experiment FooUS * Desktop,"-0.22%[0.07, 0.36] %","--0.66%[-1.46, 0.14] %","--0.87%[-0.94, -0.80] %"
The Answer42US * Mobile,"--0.25%[-0.56, 0.05] %","-0.90%[0.80, 1.00] %","-0.46%[-0.22, 1.15] %"
222US * Mobile,"-0.42%[-0.25, 1.08] %","--0.96%[-1.27, -0.65] %","-0.94%[0.42, 1.46] %"
Bold666US * Mobile,"--0.14%[-0.73, 0.46] %","--0.42%[-1.34, 0.50] %","-0.22%[0.14, 0.31] %"
Experiment FooUS * Mobile,"-0.03%[-0.33, 0.39] %","-0.18%[-0.10, 0.47] %","--0.91%[-1.45, -0.36] %"
The Answer42US * Tablet,"--0.88%[-0.92, -0.85] %","-0.73%[-0.18, 1.64] %","-0.20%[-0.06, 0.46] %"
222US * Tablet,"--0.63%[-1.41, 0.14] %","--0.39%[-1.33, 0.55] %","-0.05%[-0.85, 0.94] %"


In [ ]:
# You can hide the null control values.
confidence_interval_display.render(
    test_df2, show_control=False, hide_null_ctrl=True)

Dimensions,Click,Latency,a very very very looooooooooong metric
The Answer42US * Desktop,"0.20%[-0.24, 0.64] %","-0.69%[-0.81, -0.57] %","-0.69%[-1.18, -0.19] %"
222US * Desktop,"0.66%[0.12, 1.21] %","-0.58%[-0.76, -0.39] %","-0.64%[-1.61, 0.33] %"
Bold666US * Desktop,"-0.72%[-0.92, -0.53] %","-0.42%[-0.46, -0.37] %","-0.27%[-0.59, 0.06] %"
Experiment FooUS * Desktop,"0.22%[0.07, 0.36] %","-0.66%[-1.46, 0.14] %","-0.87%[-0.94, -0.80] %"
The Answer42US * Mobile,"-0.25%[-0.56, 0.05] %","0.90%[0.80, 1.00] %","0.46%[-0.22, 1.15] %"
222US * Mobile,"0.42%[-0.25, 1.08] %","-0.96%[-1.27, -0.65] %","0.94%[0.42, 1.46] %"
Bold666US * Mobile,"-0.14%[-0.73, 0.46] %","-0.42%[-1.34, 0.50] %","0.22%[0.14, 0.31] %"
Experiment FooUS * Mobile,"0.03%[-0.33, 0.39] %","0.18%[-0.10, 0.47] %","-0.91%[-1.45, -0.36] %"
The Answer42US * Tablet,"-0.88%[-0.92, -0.85] %","0.73%[-0.18, 1.64] %","0.20%[-0.06, 0.46] %"
222US * Tablet,"-0.63%[-1.41, 0.14] %","-0.39%[-1.33, 0.55] %","0.05%[-0.85, 0.94] %"


In [ ]:
# You can order and slice the metrics.
confidence_interval_display.render(test_df, metric_order=['Latency', 'Click'])

Dimensions,Latency,Click
The Answer42US * Desktop,"40.0000-0.69%[-0.81, -0.57] %","158.00000.20%[-0.24, 0.64] %"
222US * Desktop,"39.7699-0.58%[-0.76, -0.39] %","159.05050.66%[0.12, 1.21] %"
Bold666US * Desktop,"39.8337-0.42%[-0.46, -0.37] %","156.8608-0.72%[-0.92, -0.53] %"
Experiment FooUS * Desktop,"39.7364-0.66%[-1.46, 0.14] %","158.33980.22%[0.07, 0.36] %"
The Answer42US * Mobile,"28.00000.90%[0.80, 1.00] %","137.0000-0.25%[-0.56, 0.05] %"
222US * Mobile,"27.7315-0.96%[-1.27, -0.65] %","137.57010.42%[-0.25, 1.08] %"
Bold666US * Mobile,"27.8831-0.42%[-1.34, 0.50] %","136.8135-0.14%[-0.73, 0.46] %"
Experiment FooUS * Mobile,"28.05180.18%[-0.10, 0.47] %","137.03900.03%[-0.33, 0.39] %"
The Answer42US * Tablet,"-20.10000.73%[-0.18, 1.64] %","-10.0000-0.88%[-0.92, -0.85] %"
222US * Tablet,"-20.0213-0.39%[-1.33, 0.55] %","-9.9367-0.63%[-1.41, 0.14] %"


In [ ]:
# You can customize the order of dimension slices.
confidence_interval_display.render(
    test_df,
    sort_by=[{
        'column': 'Dim_2',
        'order': ['Mobile', 'Desktop', 'Tablet']
    }])

Dimensions,Click,Latency,a very very very looooooooooong metric
The Answer42US * Mobile,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
Bold666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
Experiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
The Answer42US * Desktop,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
Bold666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
Experiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
The Answer42US * Tablet,"-10.0000-0.88%[-0.92, -0.85] %","-20.10000.73%[-0.18, 1.64] %","33.00000.20%[-0.06, 0.46] %"
222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# You can sort by values of a metric. The supported value columns are 'Value',
# 'Ratio', 'CI_Lower', 'CI_Upper'. See the doc of get_formatted_df() also.
confidence_interval_display.render(
    test_df,
    sort_by=[{
        'column': ('Ratio', 'Click'),
        'ascending': False
    }])

Dimensions,Click,Latency,a very very very looooooooooong metric
Experiment FooUS * Tablet,"-10.08980.90%[-0.09, 1.88] %","-20.28720.93%[0.16, 1.70] %","33.20350.62%[0.42, 0.82] %"
222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
Experiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
The Answer42US * Desktop,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
Experiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
Bold666US * Tablet,"-9.9912-0.09%[-0.48, 0.30] %","-20.21460.57%[0.30, 0.84] %","32.8018-0.60%[-1.43, 0.23] %"
Bold666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
The Answer42US * Mobile,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# Of course you can sort both by dimesnion and by values.
confidence_interval_display.render(
    test_df,
    sort_by=[{
        'column': 'Dim_2',
        'order': ['Mobile', 'Desktop', 'Tablet']
    }, {
        'column': ('CI_Lower', 'Click'),
    }])

Dimensions,Click,Latency,a very very very looooooooooong metric
Bold666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
The Answer42US * Mobile,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
Experiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
Bold666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
The Answer42US * Desktop,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
Experiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"
The Answer42US * Tablet,"-10.0000-0.88%[-0.92, -0.85] %","-20.10000.73%[-0.18, 1.64] %","33.00000.20%[-0.06, 0.46] %"


In [ ]:
pre_agg_df = confidence_interval_display.render(
    test_df, return_pre_agg_df=True)
mask = pre_agg_df.applymap(lambda c: bool(
    isinstance(c, list) and c[-1] and c[-2] and c[-1] - c[-2] < 1))
formatted_df = confidence_interval_display.render(
    test_df, return_formatted_df=True)
formatted_df.mask(
    mask,
    formatted_df.applymap('<div class="tight-ci">{}</div>'.format),
    inplace=True)

extra_css = '.tight-ci {border-color: purple; border-style: dotted;}'
confidence_interval_display.display_formatted_df(formatted_df, extra_css)

Dimensions,Click,Latency,a very very very looooooooooong metric
The Answer42US * Desktop,"158.00000.20%[-0.24, 0.64] %","40.0000-0.69%[-0.81, -0.57] %","6.6667-0.69%[-1.18, -0.19] %"
222US * Desktop,"159.05050.66%[0.12, 1.21] %","39.7699-0.58%[-0.76, -0.39] %","6.6242-0.64%[-1.61, 0.33] %"
Bold666US * Desktop,"156.8608-0.72%[-0.92, -0.53] %","39.8337-0.42%[-0.46, -0.37] %","6.6488-0.27%[-0.59, 0.06] %"
Experiment FooUS * Desktop,"158.33980.22%[0.07, 0.36] %","39.7364-0.66%[-1.46, 0.14] %","6.6087-0.87%[-0.94, -0.80] %"
The Answer42US * Mobile,"137.0000-0.25%[-0.56, 0.05] %","28.00000.90%[0.80, 1.00] %","999.90000.46%[-0.22, 1.15] %"
222US * Mobile,"137.57010.42%[-0.25, 1.08] %","27.7315-0.96%[-1.27, -0.65] %","1009.29730.94%[0.42, 1.46] %"
Bold666US * Mobile,"136.8135-0.14%[-0.73, 0.46] %","27.8831-0.42%[-1.34, 0.50] %","1002.13680.22%[0.14, 0.31] %"
Experiment FooUS * Mobile,"137.03900.03%[-0.33, 0.39] %","28.05180.18%[-0.10, 0.47] %","990.8299-0.91%[-1.45, -0.36] %"
The Answer42US * Tablet,"-10.0000-0.88%[-0.92, -0.85] %","-20.10000.73%[-0.18, 1.64] %","33.00000.20%[-0.06, 0.46] %"
222US * Tablet,"-9.9367-0.63%[-1.41, 0.14] %","-20.0213-0.39%[-1.33, 0.55] %","33.01630.05%[-0.85, 0.94] %"


In [ ]:
# Test for no dimensions.
df_no_dim = test_df.groupby(['Experiment_Id', 'Metric']).mean().reset_index()
confidence_interval_display.render(df_no_dim, ctrl_id=42)

Dimensions,Click,Latency,a very very very looooooooooong metric
Control42,95.0000,15.9667,346.5222
Not Control222,"95.56130.15%[-0.51, 0.81] %","15.8267-0.64%[-1.12, -0.16] %","349.64590.12%[-0.68, 0.91] %"
Not Control666,"94.5610-0.31%[-0.71, 0.08] %","15.8341-0.09%[-0.50, 0.33] %","347.1958-0.21%[-0.63, 0.20] %"
Not ControlExperiment Foo,"95.09640.38%[-0.11, 0.88] %","15.83370.15%[-0.47, 0.77] %","343.5474-0.39%[-0.66, -0.11] %"
